In [1]:
import boto3
import os, zipfile
import shutil
import json, pandas as pd

In [2]:
client = boto3.client('s3', aws_access_key_id='xxx',
    aws_secret_access_key='xxx')

#Conexión a la API de S3

In [3]:
BUCKET = 'xxx'

In [4]:
# ANTES DE NADA, DESCARGAR ZIP ETIQUETAS
# En CVAT exportar como YOLO, sin incluir imágenes, y que lo ponga en un dir de S3. 
# Es decir, nombrar el zip como directorio remoto imagenesEPI/resultadoscvat/train, por ejemplo)

## Descarga y descompresión de carpeta .zip

In [5]:
tmpfolder='temporal'
resfolder='res'
dirs3='imagenesEPI/resultadoscvat/'
filename='fotos_arneses2.zip'

In [6]:
os.mkdir(tmpfolder)
os.mkdir(resfolder)
# Crea directorio temporal y de resultados

In [7]:
client.download_file(BUCKET,dirs3+filename,tmpfolder+'/'+filename)
# Descarga el zip de S3

In [8]:
with zipfile.ZipFile(tmpfolder+'/'+filename,"r") as zip_ref:
    zip_ref.extractall(resfolder)
# Descomprime el zip

In [9]:
shutil.rmtree(tmpfolder)
#borra carpeta temporal

In [ ]:
client.delete_object( Bucket=BUCKET,Key=dirs3+filename)
#borra zip de s3    

## Procesado de etiquetas

In [10]:
tagFolder='res/obj_train_data/imagenesEPI/sin_etiquetar/2023_12_arnes/'
# El fichero .zip, que se descarga desde CVAT con las etiquetas, tiene un directorio obj_train_data que por dentro se estructura de la misma forma que le bucket de S3 desde donde se subieron las imágenes a CVAT

In [11]:
files=os.listdir(tagFolder)


In [12]:
res=[]
for file in files:
    if file.endswith('.txt'):
        if os.stat(tagFolder+file).st_size >0:
            res.append(file)    

# Crea lista de ficheros con etiquetas (solo añade los ficheros que tienen etiquetas, ya que desde CVAT se exportan todos los ficheros, aunque la imagen no se haya etiquetado)

In [13]:
len(res)

616

In [14]:
TEST_PCT=20
VAL_PCT=15

# Define porcentaje de archivos que irán a test y validación

In [16]:
test_size=round(len(res)*TEST_PCT/100)

In [17]:
val_size=round(len(res)*VAL_PCT/100)

In [18]:
train_size=len(res)-test_size-val_size

In [ ]:
# En las celdas anteriores se ha definido el tamaño de los conjuntos de train, test y validación

In [19]:
import random
random.shuffle(res)
# Mezcla aleatoriamente los ficheros de etiquetas

In [21]:
train=res[:train_size]
val=res[train_size:train_size+val_size]
test=res[train_size+val_size:]
# Divide ficheros en las carpetas de train, test y validación

## Creación de carpetas "labels" e "images"

In [24]:
dsFolder='imagenesEPI/datasetarneses1completo/'

In [25]:
for file in train:
    client.upload_file(tagFolder+file,BUCKET, dsFolder+'labels/train/'+file)
    client.copy_object(CopySource=BUCKET+'/imagenesEPI/sin_etiquetar/2023_12_arnes/'+file[:-3]+'jpeg', Bucket=BUCKET,
    Key=dsFolder+'images/train/'+file[:-3]+'jpeg'
)
    
# Crea carpetas, en S3, de train dentro de images y labels y sube las imágenes y etiquetas de train

In [26]:
for file in test:
    client.upload_file(tagFolder+file,BUCKET, dsFolder+'labels/test/'+file)
    client.copy_object(CopySource=BUCKET+'/imagenesEPI/sin_etiquetar/2023_12_arnes/'+file[:-3]+'jpeg', Bucket=BUCKET,
    Key=dsFolder+'images/test/'+file[:-3]+'jpeg'
)
    
# Crea carpetas, en S3, de test dentro de images y labels y sube las imágenes y etiquetas de test

In [27]:
for file in val:
    client.upload_file(tagFolder+file,BUCKET, dsFolder+'labels/val/'+file)
    client.copy_object(CopySource=BUCKET+'/imagenesEPI/sin_etiquetar/2023_12_arnes/'+file[:-3]+'jpeg', Bucket=BUCKET,
    Key=dsFolder+'images/val/'+file[:-3]+'jpeg'
)
    
# Crea carpetas, en S3, de val dentro de images y labels y sube las imágenes y etiquetas de val

In [ ]:
#bajar dataset

## Descarga dataset estructurado a local (instancia SageMaker)

In [4]:
dsFolder='imagenesEPI/datasetarneses1completo/'

In [5]:
localdsfolder='dataset_fotosarneses2_completo'

In [6]:
os.mkdir(localdsfolder)
#crea directorio, en local, donde se guarda el dataset

In [7]:
keys = []
dirs = []
next_token = ''
base_kwargs = {
    'Bucket':BUCKET,
    'Prefix':dsFolder,
}

In [8]:
exfiles=[]
for root, subdirs, files in os.walk(localdsfolder+'/'):
    if len(files)>0:
        print(root)
        print(len(files))
        for file in files:
            exfiles.append(root[8:]+'/'+file)

#  Recorre recursivamente el directorio especificado y recopila información sobre su contenido

In [9]:
while next_token is not None:
    kwargs = base_kwargs.copy()
    if next_token != '':
        kwargs.update({'ContinuationToken': next_token})
    results = client.list_objects_v2(**kwargs)
    contents = results.get('Contents')
    for i in contents:
        k = i.get('Key')
        if k[-1] != '/':
            keys.append(k)
    next_token = results.get('NextContinuationToken')

    # Lista todos los objetos del bucket de Amazon S3 bajo un prefijo específico (dsfolder), excluyendo los "directorios" (claves que terminan con /)

In [10]:
for k in keys:
    dest_pathname = os.path.join(localdsfolder, k[len(dsFolder):])
    if not os.path.exists(os.path.dirname(dest_pathname)):
        os.makedirs(os.path.dirname(dest_pathname))
    if k[20:] not in exfiles:
        client.download_file(BUCKET, k, dest_pathname)
    else:
        print(k)

# Descarga archivos desde el bucket S3 a un directorio local, asegurándose de que solo se descarguen los archivos que no están en la lista de archivos ya existentes (exfiles)

In [ ]:
# El resultado de este proceso es el dataset con las imágenes y etiquetas estructurados en carpetas de train, test y val, dentro de sus respectivas carpetas (images y labels), en el entorno local (instanncia de SageMaker)
